In [1]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np

# Basic utility functions: 

In [2]:
import os, contextlib
import pathlib
import subprocess

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [3]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="None",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        if folder == "None":
            file_loc = pathlib.Path.cwd()
            
        else:
            file_loc = pathlib.Path.cwd() / folder
            
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

# Skeleton Utility Functions:

In [4]:
def filter_distance_from_skeleton( skeleton_vertices,
                            child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=500,
                             significance_threshold=500,
                             n_sample_points=3, #currently this does not do anything
                             print_flag=False,
                             return_verts_faces=False):
    """
    Will return all of the significant mesh pieces that are far enough away from the 
    mesh skeleton
    """
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(skeleton_vertices)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    #gets the faces after has the vertices
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = new_submesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []
    
    #arrange the significant pieces from largest to smallest
    x = [len(k.vertices) for k in significant_pieces]
    sorted_indexes = sorted(range(len(x)), key=lambda k: x[k])
    sorted_indexes = sorted_indexes[::-1]
    sorted_significant_pieces = [significant_pieces[k] for k in sorted_indexes]
    
    return sorted_significant_pieces

In [5]:
"""
Extra skeleton functions:
"""
import networkx as nx
import time 
import numpy as np
import trimesh
import random

def generate_surface_skeleton(vertices, faces, surface_samples):
    
    #return surface_with_poisson_skeleton,path_length
    
    current_mesh = trimesh.Trimesh(vertices=vertices,
                                  faces = faces)


    start_time = time.time()

    # test on a sphere mesh
    mesh = current_mesh

    ga = nx.from_edgelist(mesh.edges)

    if surface_samples<len(vertices):
        k = surface_samples
    else:
        k = len(vertices)
    sampled_nodes = random.sample(ga.nodes, k)


    lp_end_list = []
    lp_magnitude_list = []

    for s in sampled_nodes: 
        sp_dict = nx.single_source_shortest_path_length(ga,s)

        list_keys = list(sp_dict.keys())
        longest_path_node = list_keys[len(list_keys)-1]
        longest_path_magnitude = sp_dict[longest_path_node]


        lp_end_list.append(longest_path_node)
        lp_magnitude_list.append(longest_path_magnitude)

    #construct skeleton from shortest path
    final_start = sampled_nodes[np.argmax(lp_magnitude_list)]
    final_end = sampled_nodes[np.argmax(lp_end_list)]

    node_list = nx.shortest_path(ga,final_start,final_end)
    if len(node_list) < 2:
        print("node_list len < 2 so returning empty list")
        return np.array([])
    #print("node_list = " + str(node_list))
    final_skeleton = mesh.vertices[np.vstack([node_list[:-1],node_list[1:]]).T]
    print(f"   Final Time for surface skeleton with sample size = {k} = {time.time() - start_time}")

    return final_skeleton

def save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path):
    """
    surface_with_poisson_skeleton (np.array) : nx2 matrix with the nodes
    """
    first_node = surface_with_poisson_skeleton[0][0]
    end_nodes =  surface_with_poisson_skeleton[:,1]
    
    skeleton_to_write = str(len(end_nodes) + 1) + " " + str(first_node[0]) + " " +  str(first_node[1]) + " " +  str(first_node[2])
    
    for node in end_nodes:
        skeleton_to_write +=  " " + str(node[0]) + " " +  str(node[1]) + " " +  str(node[2])
    
    output_file = largest_mesh_path
    if output_file[-5:] != ".cgal":
        output_file += ".cgal"
        
    f = open(output_file,"w")
    f.write(skeleton_to_write)
    f.close()
    return 

#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)
    unique_skeleton_verts = bone_array_total.reshape(-1,3)

def calculate_skeleton_distance(my_skeleton):
    total_distance = np.sum(np.sqrt(np.sum((my_skeleton[:,0] - my_skeleton[:,1])**2,axis=1)))
    return float(total_distance)



"""
Testing if the read and write are correct:

import numpy as np
file_name = "107816118160698192_leftover_1-0_skeleton.cgal"
my_skeleton = read_skeleton_revised(file_name)
my_skeleton
practice_file = "test_manual_cgal.cgal"
save_skeleton_cgal(my_skeleton,practice_file)
my_skeleton_2 = read_skeleton_revised(practice_file)

"""

"""
Pseudocode:
1) Get the number of rows to be even (by doing nothing or adding the first one to the final list
2) Take every other left neuorn, 1 + Take every other right neuron


A B
B C
C D
D E
E F
F G

Turns into

A C
C E
E G
"""

def downsample_skeleton(current_skeleton):
    #print("current_skeleton = " + str(current_skeleton.shape))
    """
    Downsamples the skeleton by 50% number of edges
    """
    extra_segment = []
    if current_skeleton.shape[0] % 2 != 0:
        extra_segment = np.array([current_skeleton[0]])
        current_skeleton = current_skeleton[1:]
        #print("extra_segment = " + str(extra_segment))
        #print("extra_segment.shape = " + str(extra_segment.shape))
    else:
        #print("extra_segment = " + str(extra_segment))
        pass

    even_indices = [k for k in range(0,current_skeleton.shape[0]) if k%2 == 0]
    odd_indices = [k for k in range(0,current_skeleton.shape[0]) if k%2 == 1]
    even_verts = current_skeleton[even_indices,0,:]
    odd_verts = current_skeleton[odd_indices,1,:]

    downsampled_skeleton = np.hstack([even_verts,odd_verts]).reshape(even_verts.shape[0],2,3)
    #print("dowsampled_skeleton.shape = " + str(downsampled_skeleton.shape))
    if len(extra_segment) > 0:
        #print("downsampled_skeleton = " + str(downsampled_skeleton.shape))
        final_downsampled_skeleton = np.vstack([extra_segment,downsampled_skeleton])
    else:
        final_downsampled_skeleton = downsampled_skeleton
    return final_downsampled_skeleton

In [6]:
import ipyvolume as ipv

def graph_skeleton_and_mesh(main_mesh_verts=[],main_mesh_faces=[],
                            unique_skeleton_verts_final=[],edges_final=[]):
    """
    Graph the final result: 
    """

    ipv.figure(figsize=(15,15))
    
    if len(unique_skeleton_verts_final) > 0 and len(edges_final) > 0:
        mesh2 = ipv.plot_trisurf(unique_skeleton_verts_final[:,0], 
                                unique_skeleton_verts_final[:,1], 
                                unique_skeleton_verts_final[:,2], 
                                lines=edges_final, color='blue')

        mesh2.color = [0,0.,1,1]
        mesh2.material.transparent = True
    
    if len(main_mesh_verts) > 0 and len(main_mesh_faces) > 0:
        main_mesh = trimesh.Trimesh(vertices=main_mesh_verts,faces=main_mesh_faces)

        mesh3 = ipv.plot_trisurf(main_mesh.vertices[:,0],
                               main_mesh.vertices[:,1],
                               main_mesh.vertices[:,2],
                               triangles=main_mesh.faces)
        mesh3.color = [0.,1.,0.,0.2]
        mesh3.material.transparent = True


        volume_max = np.max(main_mesh.vertices,axis=0)
        volume_min = np.min(main_mesh.vertices,axis=0)

        ranges = volume_max - volume_min
        index = [0,1,2]
        max_index = np.argmax(ranges)
        min_limits = [0,0,0]
        max_limits = [0,0,0]

        buffer = 10000
        for i in index:
            if i == max_index:
                min_limits[i] = volume_min[i] - buffer
                max_limits[i] = volume_max[i] + buffer 
                continue
            else:
                difference = ranges[max_index] - ranges[i]
                min_limits[i] = volume_min[i] - difference/2  - buffer
                max_limits[i] = volume_max[i] + difference/2 + buffer

        #ipv.xyzlim(-2, 2)
        ipv.xlim(min_limits[0],max_limits[0])
        ipv.ylim(min_limits[1],max_limits[1])
        ipv.zlim(min_limits[2],max_limits[2])

    ipv.style.set_style_light()
    ipv.style.box_off()
    ipv.style.axes_off()

    ipv.show()


# Main recursive skeleton functions:

In [7]:
def recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth,
                            skeletonization_type,
                            surface_samples=200,
                            max_depth_cgal = 8,
                            max_depth_surface = 10,
                            min_skeleton_distance = 30,
                           boolean_distance_threshold=1000,
                            boolean_significance_threshold=1000,
                            boolean_n_sample_points=10,
                            boolean_print_flag = False,
                           largest_mesh_significant=3000,
                           list_of_meshes=[]):
    """
    Function that will run on a single mesh or list of mesh and will 
    run the skeletonization process where will skeletonize as many segments
    as possible through a recursive algorithm
    
    """
    print("\n\n--------Beginning of loop: current_depth = " + str(current_depth) + "--------")
    
    if len(list_of_meshes):
        mesh_pieces=list_of_meshes

        print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
        # check if the number of 
        for j,piece in enumerate(mesh_pieces):
            new_current_name = current_name + "-" + str(j)
            print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
            recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                                   directory=directory,current_base_path=current_base_path,
                                    mesh_base_path=mesh_base_path,folder_name=folder_name,
                                   current_depth=current_depth,
                                    skeletonization_type=skeletonization_type,
                                    surface_samples=surface_samples,
                                    max_depth_cgal = max_depth_cgal,
                                    max_depth_surface = max_depth_surface,
                                    min_skeleton_distance = min_skeleton_distance,
                                   boolean_distance_threshold=boolean_distance_threshold,
                                    boolean_significance_threshold=boolean_significance_threshold,
                                    boolean_n_sample_points=boolean_n_sample_points,
                                    boolean_print_flag = boolean_print_flag,
                                   largest_mesh_significant=largest_mesh_significant)
        return
        
    else:
        print(f"\n----------Starting {current_name}----------")
        global_time = time.time()
        #print out the off file
        if current_depth >= max_depth_surface:
            print(f"\n\n*********Reamched maximum depth of {max_depth_surface} so returning********\n\n")
            return
        if skeletonization_type== "cgal" and current_depth >= max_depth_cgal:
            print("     Reached max depth cgal")

            skeletonization_type = "surface_with_poisson"
            current_name += "_sWp"

            recursive_skeletonizing(verts,faces,current_name,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=current_depth,
                            skeletonization_type=skeletonization_type,
                            surface_samples=surface_samples,
                            max_depth_cgal = max_depth_cgal,
                            max_depth_surface = max_depth_surface,
                            min_skeleton_distance = min_skeleton_distance,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=boolean_n_sample_points,
                            boolean_print_flag = boolean_print_flag,
                           largest_mesh_significant=largest_mesh_significant)
            return




        if skeletonization_type == "surface_without_poisson":
            print("Inside WITHOUT poisson generation")
            
            """
            generate all of the mesh pieces if this is the first layer
            """
            if current_depth == 1:
                current_depth += 1
                #generate the mesh pieces
                new_submesh = trimesh.Trimesh(vertices=verts,faces=faces)
                mesh_pieces = new_submesh.split(only_watertight=False)
    
                if boolean_print_flag:
                    print(f"There were {len(mesh_pieces)} pieces after mesh split")

                significant_pieces = [m for m in mesh_pieces if len(m.faces) > boolean_significance_threshold]

                if boolean_print_flag:
                    print(f"There were {len(significant_pieces)} pieces found after size threshold")
                if len(significant_pieces) <=0:
                    print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
                    return []

                #arrange the significant pieces from largest to smallest
                x = [len(k.vertices) for k in significant_pieces]
                sorted_indexes = sorted(range(len(x)), key=lambda k: x[k])
                sorted_indexes = sorted_indexes[::-1]
                sorted_significant_pieces = [significant_pieces[k] for k in sorted_indexes]
                
                
                mesh_pieces = sorted_significant_pieces

                print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
                # check if the number of 
                for j,piece in enumerate(mesh_pieces):
                    new_current_name = current_name + "-" + str(j)
                    print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
                    recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                                           directory=directory,current_base_path=current_base_path,
                                            mesh_base_path=mesh_base_path,folder_name=folder_name,
                                           current_depth=current_depth,
                                            skeletonization_type=skeletonization_type,
                                            surface_samples=surface_samples,
                                            max_depth_cgal = max_depth_cgal,
                                            max_depth_surface = max_depth_surface,
                                            min_skeleton_distance = min_skeleton_distance,
                                           boolean_distance_threshold=boolean_distance_threshold,
                                            boolean_significance_threshold=boolean_significance_threshold,
                                            boolean_n_sample_points=boolean_n_sample_points,
                                            boolean_print_flag = boolean_print_flag,
                                           largest_mesh_significant=largest_mesh_significant)
                return
            
            else:
            
                surface_without_poisson_skeleton = generate_surface_skeleton(verts,
                                                                              faces=faces,
                                                                              surface_samples = surface_samples)
                if len(surface_without_poisson_skeleton)>2:
                    surface_without_poisson_skeleton = downsample_skeleton(surface_without_poisson_skeleton)
                if len(surface_without_poisson_skeleton) <= 0 :
                    print("Returning because the skeleton returned was of size 0")
                    return
                #print(f"Path length for surface skeleton WITHOUT poisson = {path_length}")

                largest_mesh_path = mesh_base_path + current_name

                #save off the skeleton
                save_skeleton_cgal(surface_without_poisson_skeleton,largest_mesh_path + "_skeleton")


        else:
            print("Inside poisson generation")

            initial_output_path = mesh_base_path + current_name

            write_Whole_Neuron_Off_file(vertices=verts, 
                                        triangles=faces,
                                       path_and_filename=initial_output_path)


            # do the poisson surface reconstruction

            script_name = "poisson_working_meshlab.mls"

            input_file_base = initial_output_path
            output_file = input_file_base + "_poisson"
            meshlab_script_path_and_name =current_base_path + script_name

            start_time = time.time()
            print("     Starting Screened Poisson")
            meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                               output_path_and_filename=output_file + ".off",
                                                             meshlab_script=meshlab_script_path_and_name)
            print(f"     Total_time for Screended Poisson = {time.time() - start_time}")

            #2) Filter away for largest_poisson_piece:

            new_mesh = trimesh.load_mesh(output_file + ".off")
            mesh_splits = new_mesh.split(only_watertight=False)
            mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
            largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
            largest_mesh = mesh_splits[largest_index][0]

            print("len(largest_mesh.vertices) = " + str(len(largest_mesh.vertices)))
            #4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
            if len(largest_mesh.vertices) < largest_mesh_significant:
                print(current_name + " largest mesh not significant AFTER POISSON RECONSTRUCTION")
                print("trying surface skeletonization without Poisson")

                """3) if no significant poisson piece --> do surface_with_poisson process
                    change the name slightly
                    change the skeletonization type"""

                skeletonization_type = "surface_without_poisson"
                current_name += "_sWOp"

                recursive_skeletonizing(verts,faces,current_name,
                                directory,current_base_path,mesh_base_path,folder_name,
                                current_depth=current_depth,
                                skeletonization_type=skeletonization_type,
                                surface_samples=surface_samples,
                                max_depth_cgal = max_depth_cgal,
                                max_depth_surface = max_depth_surface,
                                min_skeleton_distance = min_skeleton_distance,
                               boolean_distance_threshold=boolean_distance_threshold,
                                boolean_significance_threshold=boolean_significance_threshold,
                                boolean_n_sample_points=boolean_n_sample_points,
                                boolean_print_flag = boolean_print_flag,
                               largest_mesh_significant=largest_mesh_significant)
                return 

            if skeletonization_type == "surface_with_poisson":
                surface_with_poisson_skeleton = generate_surface_skeleton(largest_mesh.vertices,
                                                                          faces=largest_mesh.faces,
                                                                          surface_samples = surface_samples)

                if len(surface_with_poisson_skeleton) <= 0 :
                    print("Returning because the skeleton returned was of size 0")
                    return
                #print(f"Path length for surface skeleton with poisson = {path_length}")

                largest_mesh_path = mesh_base_path + current_name

                #save off the skeleton
                save_skeleton_cgal(surface_with_poisson_skeleton,largest_mesh_path + "_skeleton")


            elif skeletonization_type == "cgal":

                #5) If significant size output the mesh
                largest_mesh_path = mesh_base_path + current_name
                write_Whole_Neuron_Off_file(vertices=largest_mesh.vertices, 
                                            triangles=largest_mesh.faces,
                                           path_and_filename=largest_mesh_path)


                #6) Run skeletonization on it:
                start_time = time.time()
                print("     Starting Calcification")
                cm.calcification(largest_mesh_path)
                print(f"     Total_time for Calcification = {time.time() - start_time}")




        """ ***** GOING TO USE THE SKELETON VERTICES INSTEAD OF THE MESH VERTICES"""


        #7) Subtract skeleton vertices from current mesh
        start_time = time.time()
        print("     Starting Mesh boolean difference")

        cgal_skeleton_file = largest_mesh_path + "_skeleton.cgal"

        try:
            returned_skeleton = read_skeleton_revised(cgal_skeleton_file)
        except OSError as e:
            print("\n\n**** No cgal file was found so returning ******\n\n")
            return
        else:
            pass
            #print("\n\n**** unknown error ocured when reading in cgal file so returning ******\n\n")
            #return


        skeletal_length = calculate_skeleton_distance(returned_skeleton)
        print(f" Skeletal Length = {skeletal_length}  ")
        # returning based on skeletal length: 
        if skeletal_length <= min_skeleton_distance:

            if skeletonization_type == "cgal":
                print("Skeleton generated was too small --> cgal skeletonization turning into surface with poisson")
                #try the surface skeletonization with poisson
                current_name += "sWp"
                skeletonization_type = "surface_with_poisson"

            elif skeletonization_type == "surface_with_poisson":
                print("Skeleton generated was too small --> surface WITH poisson turning into surface WTIHOUT poisson")
                current_name += "sWOp"
                skeletonization_type = "surface_without_poisson"
            else:
                return

            recursive_skeletonizing(verts,faces,current_name,
                        directory,current_base_path,mesh_base_path,folder_name,
                        current_depth=current_depth,
                        skeletonization_type=skeletonization_type,
                        surface_samples=surface_samples,
                        max_depth_cgal = max_depth_cgal,
                        max_depth_surface = max_depth_surface,
                        min_skeleton_distance = min_skeleton_distance,
                       boolean_distance_threshold=boolean_distance_threshold,
                        boolean_significance_threshold=boolean_significance_threshold,
                        boolean_n_sample_points=boolean_n_sample_points,
                        boolean_print_flag = boolean_print_flag,
                       largest_mesh_significant=largest_mesh_significant)
            return




        skeleton_vertices = np.vstack([returned_skeleton[:,0],returned_skeleton[:,1]])
        #getting the boolean difference

        if skeletonization_type == "cgal":
            current_boolean_distance_threshold = boolean_distance_threshold
        else:
            current_boolean_distance_threshold = 500

        mesh_pieces = filter_distance_from_skeleton( skeleton_vertices,
                                verts,
                                 faces,
                                 distance_threshold=current_boolean_distance_threshold,
                                 significance_threshold=boolean_significance_threshold,
                                 n_sample_points=boolean_n_sample_points,
                                 print_flag=boolean_print_flag,
                                 return_verts_faces=False)



        print(f"     Total_time for Mesh boolean difference = {time.time() - start_time}")

        print(f"Total time for one mesh piece skeleton = {time.time() - global_time}")

        print(f"{current_name} there were {len(mesh_pieces)} significant pieces leftover after largest mesh")

        if len(mesh_pieces) <= 0:
            print(f"{current_name} returning because 0 significant pieces")
            return 



        #increment the current depth 
        current_depth = current_depth + 1

        print(f" -->  Total significant pieces left: {[len(k.vertices) for k in mesh_pieces]}")
        # check if the number of 
        for j,piece in enumerate(mesh_pieces):
            new_current_name = current_name + "-" + str(j)
            print(f"Starting recursive call for size vertices of {len(piece.vertices)}")
            recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name,
                                   directory=directory,current_base_path=current_base_path,
                                    mesh_base_path=mesh_base_path,folder_name=folder_name,
                                   current_depth=current_depth,
                                    skeletonization_type=skeletonization_type,
                                    surface_samples=surface_samples,
                                    max_depth_cgal = max_depth_cgal,
                                    max_depth_surface = max_depth_surface,
                                    min_skeleton_distance = min_skeleton_distance,
                                   boolean_distance_threshold=boolean_distance_threshold,
                                    boolean_significance_threshold=boolean_significance_threshold,
                                    boolean_n_sample_points=boolean_n_sample_points,
                                    boolean_print_flag = boolean_print_flag,
                                   largest_mesh_significant=largest_mesh_significant)

        return

In [8]:
"""
Pseudocode for skeleton stitcher: 
-------Old method-------

1) creates a network with unique nodes and their coordinates and the edges of the graph
2) extract subgraphs: Appends the list of subgraphs
3) Iterates through all of the subgraphs
a. gets first subgraph
b. gets current coordinates of subgraph
c. strip these coordinates from all the list of entire coordinates
d. Find closest node
e. Add edge (make sure not greater than max length)
f. recompute the subgraphs
g. keep looping until only one component
4) Output the edges as a skeleton of coordinates



"""

from pykdtree.kdtree import KDTree

import scipy
def stitch_skeleton_with_degree_check_vp4(staring_edges,max_stitch_distance=18000,end_node=False):
    

    #unpacks so just list of vertices
    vertices_unpacked  = staring_edges.reshape(-1,3)

    #reduce the number of repeat vertices and convert to list
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()

    #assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
    vertices_unpacked_coefficients = np.array([unique_rows_list.index(a) for a in vertices_unpacked.tolist()])

    #reshapes the vertex list to become an edge list
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    
    #create the graph from the edges
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)

    # find the shortest distance between the two different subgraphs:
    from scipy.spatial import distance_matrix

    UG = B.to_undirected()

#     # extract subgraphs
#     sub_graphs = connected_component_subgraphs(UG)

#     subgraphs_list = []
#     for i, sg in enumerate(sub_graphs):
#         #print("subgraph {} has {} nodes".format(i, sg.number_of_nodes()))
#         #print("\tNodes:", sg.nodes(data=True))
#         #print("\tEdges:", sg.edges())
#         subgraphs_list.append(sg)

    #get all of the coordinates

    print("len_subgraphs AT BEGINNING of the loop")
    counter = 0
    print_flag = True
    while True:
        counter+= 1
        if print_flag:
            print(f"Starting Loop {counter}")
        start_time = time.time()
        """
        1) Get the indexes of the subgraph
        2) Build a KDTree from those not in the subgraph (save the vertices of these)
        3) Query against the nodes in the subgraph  and get the smallest distance
        4) Create this new edge
        
        """
        
        #1) Get the indexes of the subgraph
        n_components, labels = scipy.sparse.csgraph.connected_components(csgraph=nx.adjacency_matrix(UG), directed=False, return_labels=True)
        subgraph_components = np.where(labels==0)[0]
        #print("subgraph_components = " + str(subgraph_components))
        if len(subgraph_components) == len(UG.nodes):
            print("all graph is one component!")
            break

        outside_components = np.where(labels !=0)[0]
        #print("outside_components = " + str(outside_components))
        
        #2) Build a KDTree from those not in the subgraph (save the vertices of these)
        mesh_tree = KDTree(unique_rows[outside_components])

        
        #3) Query against the nodes in the subgraph  and get the smallest distance
        """
        Conclusion:
        Distance is of the size of the parts that are in the KDTree
        The closest nodes represent those that were queryed

        """
        distances,closest_node = mesh_tree.query(unique_rows[subgraph_components])
        min_index = np.argmin(distances)
        #closest_subgraph_node = subgraph_components[closest_node[min_index]]
        #closest_outside_node = outside_components[min_index]
        
        
#         print("distances = " + str(distances))
#         print("closest_node = " + str(closest_node))
#         print("Closest distance = " + str(distances[min_index]))
        closest_outside_node = outside_components[closest_node[min_index]]
        closest_subgraph_node = subgraph_components[min_index]
        
        #get the edge distance of edge about to create:
        
#         graph_coordinates=nx.get_node_attributes(UG,'coordinates')
#         prospective_edge_length = np.linalg.norm(np.array(graph_coordinates[closest_outside_node])-np.array(graph_coordinates[closest_subgraph_node]))
#         print(f"Edge distance going to create = {prospective_edge_length}")
        

        #4) Create this new edge
        UG.add_edge(closest_subgraph_node,closest_outside_node)
        
        

        if print_flag:
            print(f"Total Time for loop = {time.time() - start_time}")


    # get the largest subgraph!!! in case have missing pieces

    #add all the new edges to the 

    total_coord = nx.get_node_attributes(UG,'coordinates')

    current_coordinates = np.array(list(total_coord.values()))

    return current_coordinates[UG.edges()]


"""
For finding the distances of the skeleton
"""
def find_skeleton_distance(example_edges):
    total_distance = np.sum([np.linalg.norm(a-b) for a,b in example_edges])
    return total_distance

from scipy.spatial import distance

def find_skeleton_distance_scipy(example_edges):
    total_distance = np.sum([distance.euclidean(a, b) for a,b in example_edges])
    return total_distance

In [9]:
import calcification_Module as cm #module that allows for calcification
import time
import os
import pathlib

def calculate_skeleton_edges(input_verts,input_faces,segment_id,
                            boolean_distance_threshold=1000,
                                   boolean_significance_threshold=300,
                                   surface_samples=200,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=300,
                            skeletonization_type="surface_without_poisson",
                            list_of_meshes=[],
                            delete_after_finishing = False,
                            stitch_edges_flag=True):
    
    """
    Function that will oversee the whole skeletonization process:
    1) first generate all of the lone skeletal pieces
    2) Then call the stitching function that will piece all of the lone skeletal pieces together
    """
    
    original_mesh = trimesh.Trimesh(vertices=input_verts,faces=input_faces)
    name2 = segment_id
    
    
    directory = "./" + name2
    if not os.path.exists(directory):
        os.makedirs(directory)


    current_base_path = str(pathlib.Path.cwd()) + "/"
    mesh_base_path = current_base_path + str(name2) + "/"
    folder_name = str(name2)

    current_layer = 1
    
    
    recursive_skeletonizing(original_mesh.vertices,original_mesh.faces,name2,
                            directory,current_base_path,mesh_base_path,folder_name,
                            current_depth=1,
                            skeletonization_type=skeletonization_type,
                            surface_samples=surface_samples,
                            max_depth_cgal = 4,
                            max_depth_surface = 6,
                            min_skeleton_distance = 10,
                           boolean_distance_threshold=boolean_distance_threshold,
                            boolean_significance_threshold=boolean_significance_threshold,
                            boolean_n_sample_points=surface_samples,
                            boolean_print_flag = boolean_print_flag,
                            list_of_meshes=list_of_meshes
                           )
    
    
    
    
    cgal_skeleton_file_list = []
    
    for file in os.listdir(name2):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")
    
    
    if stitch_edges_flag:
        bone_array_total = np.vstack([read_skeleton_revised(k) 
                                      for k in cgal_skeleton_file_list])

        unique_skeleton_verts = bone_array_total.reshape(-1,3)
        edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


        total_edges = np.array([])

        for k in cgal_skeleton_file_list:
            bone_array = read_skeleton_revised(k) 

            #add the skeleton edges to the total edges
            if not total_edges.any():
                total_edges = bone_array
            else:
                total_edges = np.vstack([total_edges,bone_array])

        total_edges_stitched = stitch_skeleton_with_degree_check_vp4(total_edges)

        unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
        edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

        if delete_after_finishing:
            #erase the folder with all of the data in it
            import shutil
            shutil.rmtree(directory)

        #return the edges
        return edges,unique_skeleton_verts

In [10]:
def combined_run_stitching(folder_list=[],delete_after_finishing=False):
    if len(folder_list) <= 0:
        raise Exception("There were no folders listed in the parameters")
        
    cgal_skeleton_file_list = []
    for directory in folder_list:

        for file in os.listdir(directory):
            if file.endswith(".cgal"):
                cgal_skeleton_file_list.append(str(os.path.join(directory, file)))
        if len(cgal_skeleton_file_list) <= 0:
            #check to see if cgal list is empty
            raise Exception("There were no cgal files generated in process")


    bone_array_total = np.vstack([read_skeleton_revised(k) 
                                  for k in cgal_skeleton_file_list])

    unique_skeleton_verts = bone_array_total.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


    total_edges = np.array([])

    for k in cgal_skeleton_file_list:
        bone_array = read_skeleton_revised(k) 

        #add the skeleton edges to the total edges
        if not total_edges.any():
            total_edges = bone_array
        else:
            total_edges = np.vstack([total_edges,bone_array])

    total_edges_stitched = stitch_skeleton_with_degree_check_vp4(total_edges)

    unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
    edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

    if delete_after_finishing:
        #erase the folder with all of the data in it
        import shutil
        shutil.rmtree(directory)

    #return the edges
    return edges,unique_skeleton_verts

# ----------  Running the Skeletonization ------------

# Round 1

In [11]:
# Loading in the neuron mesh and assigning the id
import trimesh
main_mesh = trimesh.load_mesh("Louise.off")
segment_id = "Louise"

import time
total_start_time = time.time()


In [12]:
# Running the first pass that does the cgal skeletonization
calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,segment_id,
                                   boolean_distance_threshold=2000,
                                   boolean_significance_threshold=1500,
                                   surface_samples=200,
                                   boolean_print_flag=True,
                                   largest_mesh_significant=500,
                                    skeletonization_type="cgal",
                                    stitch_edges_flag=False)
print(f"Total time for calculating skeleton = {time.time() - start_time}")



--------Beginning of loop: current_depth = 1--------

----------Starting Louise----------
Inside poisson generation
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_vp2_Surface/Louise/Louise.off -o /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_vp2_Surface/Louise/Louise_poisson.off -s /notebooks3/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum/Platinum_Skeletonization_vp2_Surface/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 43.51685047149658


face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


len(largest_mesh.vertices) = 17096
     Starting Calcification
     Total_time for Calcification = 1.5901110172271729
     Starting Mesh boolean difference
 Skeletal Length = 563569.6104736524  
Total time for KDTree creation and queries: 0.6412792205810547
Original number vertices in child mesh = 639941
Number of vertices after distance threshold applied =  560393
Total time for finding faces after distance threshold applied: 6.944209337234497
------Starting the mesh filter for significant outside pieces-------
There were 1519 pieces after mesh split
There were 52 pieces found after size threshold
     Total_time for Mesh boolean difference = 10.281481504440308
Total time for one mesh piece skeleton = 73.28206753730774
Louise there were 52 significant pieces leftover after largest mesh
 -->  Total significant pieces left: [58504, 35317, 34501, 31105, 25951, 22205, 21189, 19275, 17883, 16976, 15461, 15088, 14298, 13945, 12435, 12347, 12277, 11525, 11523, 10474, 10209, 9473, 9344, 9086,

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

len(largest_mesh.vertices) = 112730
     Starting Calcification
     Total_time for Calcification = 14.004697322845459
     Starting Mesh boolean difference
 Skeletal Length = 397082.78467253275  
Total time for KDTree creation and queries: 0.021847963333129883
Original number vertices in child mesh = 58504
Number of vertices after distance threshold applied =  24659
Total time for finding faces after distance threshold applied: 0.6419456005096436
------Starting the mesh filter for significant outside pieces-------
There were 26 pieces after mesh split
There were 2 pieces found after size threshold
     Total_time for Mesh boolean difference = 0.8076627254486084
Total time for one mesh piece skeleton = 38.67940807342529
Louise-0 there were 2 significant pieces leftover after largest mesh
 -->  Total significant pieces left: [24130, 1026]
Starting recursive call for size vertices of 24130


--------Beginning of loop: current_depth = 3--------

----------Starting Louise-0-0----------
Ins

...

face_normals all zero, ignoring!


     Total_time for Screended Poisson = 6.781529903411865
len(largest_mesh.vertices) = 506
Louise-49 largest mesh not significant AFTER POISSON RECONSTRUCTION
trying surface skeletonization without Poisson


--------Beginning of loop: current_depth = 2--------

----------Starting Louise-49_sWOp----------
Inside WITHOUT poisson generation
   Final Time for surface skeleton with sample size = 200 = 1.1726305484771729
     Starting Mesh boolean difference
 Skeletal Length = 9867.998100769635  
Total time for KDTree creation and queries: 0.0021600723266601562
Original number vertices in child mesh = 776
Number of vertices after distance threshold applied =  553
Total time for finding faces after distance threshold applied: 0.010172367095947266
------Starting the mesh filter for significant outside pieces-------
There were 2 pieces after mesh split
There were 0 pieces found after size threshold
THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold
     Total_time for Mesh boolea

face_normals all zero, ignoring!


     Total_time for Screended Poisson = 6.691048622131348
len(largest_mesh.vertices) = 1020
Louise-50 largest mesh not significant AFTER POISSON RECONSTRUCTION
trying surface skeletonization without Poisson


--------Beginning of loop: current_depth = 2--------

----------Starting Louise-50_sWOp----------
Inside WITHOUT poisson generation
   Final Time for surface skeleton with sample size = 200 = 1.1536221504211426
     Starting Mesh boolean difference
 Skeletal Length = 9978.100365644312  
Total time for KDTree creation and queries: 0.0001838207244873047
Original number vertices in child mesh = 775
Number of vertices after distance threshold applied =  448
Total time for finding faces after distance threshold applied: 0.009091854095458984
------Starting the mesh filter for significant outside pieces-------
There were 3 pieces after mesh split
There were 0 pieces found after size threshold
THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold
     Total_time for Mesh boole

NameError: name 'start_time' is not defined

In [13]:
#reading in the total skeleton vertices

import os
name2 = [segment_id]
cgal_skeleton_file_list = []

for n in name2:
    for file in os.listdir(n):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(n, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts = bone_array_total.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


# Find the difference in mesh in order to do for the second pass
returned_mesh = filter_distance_from_skeleton( unique_skeleton_verts,
                            main_mesh.vertices,
                             main_mesh.faces,
                             distance_threshold=1900,
                             significance_threshold=200,
                             n_sample_points=3, #currently this does not do anything
                             print_flag=False,
                             return_verts_faces=False)
total_leftover_mesh = trimesh.Trimesh(vertices=[],faces=[])

for rm in returned_mesh:
    total_leftover_mesh += rm

In [14]:
# visualize the first pass skeletonization
graph_skeleton_and_mesh(total_leftover_mesh.vertices,total_leftover_mesh.faces,
                       unique_skeleton_verts,edges)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Round 2

In [15]:
import time
total_start_time = time.time()

returned_mesh

segment_id_run_2 = segment_id + "_Run_2"

calculate_skeleton_edges(main_mesh.vertices,main_mesh.faces,segment_id_run_2,
                                   boolean_distance_threshold=500,
                                   boolean_significance_threshold=250,
                                   surface_samples=500,
                                   boolean_print_flag=False,
                                   largest_mesh_significant=250,
                                                      skeletonization_type="surface_without_poisson",
                                                      list_of_meshes=returned_mesh,
                                    stitch_edges_flag=False)
print(f"Total time for calculating skeleton = {time.time() - total_start_time}")



--------Beginning of loop: current_depth = 1--------
 -->  Total significant pieces left: [22818, 4675, 3205, 1712, 1479, 1429, 828, 720, 715, 714, 671, 671, 664, 646, 639, 625, 622, 605, 602, 594, 593, 572, 551, 545, 543, 516, 494, 485, 481, 480, 478, 474, 457, 455, 449, 447, 437, 429, 422, 422, 418, 413, 390, 381, 377, 375, 373, 370, 367, 367, 355, 349, 348, 338, 328, 326, 320, 312, 311, 310, 309, 306, 301, 301, 286, 269, 250, 233, 221, 221, 215, 214, 214, 193, 188, 187, 187, 166, 160, 144, 135, 125, 125, 118, 117, 115, 106]
Starting recursive call for size vertices of 22818


--------Beginning of loop: current_depth = 1--------

----------Starting Louise_Run_2-0----------
Inside WITHOUT poisson generation
 -->  Total significant pieces left: [22818]
Starting recursive call for size vertices of 22818


--------Beginning of loop: current_depth = 2--------

----------Starting Louise_Run_2-0-0----------
Inside WITHOUT poisson generation


KeyboardInterrupt: 

In [ ]:
#reading in the total skeleton vertices

import os
name2 = [segment_id_run_2]
cgal_skeleton_file_list = []

for n in name2:
    for file in os.listdir(n):
        if file.endswith(".cgal"):
            cgal_skeleton_file_list.append(str(os.path.join(n, file)))
    if len(cgal_skeleton_file_list) <= 0:
        #check to see if cgal list is empty
        raise Exception("There were no cgal files generated in process")

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts_Run_2 = bone_array_total.reshape(-1,3)
edges_Run_2 = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

In [ ]:
graph_skeleton_and_mesh(total_leftover_mesh.vertices,total_leftover_mesh.faces,
                       unique_skeleton_verts_Run_2,edges_Run_2)

In [ ]:
graph_skeleton_and_mesh(main_mesh.vertices,main_mesh.faces,
                       unique_skeleton_verts_Run_2,edges_Run_2)

# Stitching Up both Runs

In [ ]:
edges,unique_skeleton_verts = combined_run_stitching([segment_id,segment_id_run_2])

# Visualizing the Final Results: 

In [ ]:
graph_skeleton_and_mesh(main_mesh.vertices,main_mesh.faces,
                        unique_skeleton_verts,edges)

# Saving off final Run

In [ ]:
np.savez("Final_" + segment_id + "_skeleton_and_mesh_consolidated.npz",unique_skeleton_verts_final=unique_skeleton_verts,edges_final=edges)